<a href="https://colab.research.google.com/github/adampotton/MDM3-Rep-3/blob/main/logit_extract/S2_logit_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import json
import torch
import numpy as np
from google.colab import files, drive
from torch.utils.data import DataLoader, TensorDataset
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load S2 Data

In [5]:
with open('/content/drive/My Drive/Aerial Data/Both Imagery/datapoints.json', 'r') as f:
    data = json.load(f)

S2_arrays = np.load('/content/drive/My Drive/Aerial Data/Both Imagery/arrays.npy')
S2_arrays_cut = S2_arrays[[item[5] for item in data]]

### Load S2 Model

In [18]:
import torch.nn as nn

num_channels = 12
num_classes = 19

model = nn.Sequential(
    nn.Conv2d(num_channels, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),

    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64 * 1 * 1, 256),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)

state_dict = torch.load("/content/drive/My Drive/S2 Data All/training_run_1_filtered_data_19.pth")
model.load_state_dict(state_dict)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### Extract logtis

In [19]:
_, all_labels_numeric = np.unique([item[4] for item in data], return_inverse=True)
all_images_tensor = torch.tensor(S2_arrays_cut.astype(np.float32), dtype=torch.float32)
all_labels_tensor = torch.tensor(all_labels_numeric, dtype=torch.long)

all_dataset = TensorDataset(all_images_tensor, all_labels_tensor)
all_loader = DataLoader(all_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def logit_extract():
    model.eval()
    all_logits = []
    with torch.no_grad():

        for images, labels in all_loader:
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images.permute(0, 3, 1, 2))
            all_logits.append(logits)

        all_logits = torch.cat(all_logits, dim=0)

    return all_logits

all_logits = logit_extract()